<a href="https://colab.research.google.com/github/llA-Wilsonll/Simpsons_Character_Recognition/blob/master/The_Simpsons_Character_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Data

Importing the required packages

In [0]:
import os
import zipfile
import sys
import shutil
import random

Upload the kaggle json Key which allows us to download data from Kaggle.

In [3]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aaroncwilson","key":"8fd509bef31a87eec6f17681d818cf03"}'}

The following packages are used so we can save files from Colab to Google Drive and vice versa.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Downloading the Kaggle Dataset
We start by downloading the dataset prepared by alexattia on Kaggle: https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [5]:
!kaggle datasets download -d alexattia/the-simpsons-characters-dataset -p /content/

 99% 1.06G/1.08G [00:34<00:00, 36.2MB/s]
100% 1.08G/1.08G [00:34<00:00, 33.4MB/s]


We then create the directory where we want to store the data and then unzip the file we downloaded into it.

In [0]:
os.mkdir('/content/simpsons/')

In [0]:
local_zip = '/content/the-simpsons-characters-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/simpsons')
zip_ref.close()

To get an idea of how many images we have of each character we run the below code.

In [75]:
characters = os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset')
print('There are ' + str(len(characters)) + ' characters.')

# Creating a dictionary to store the number of images per character
character_observations_dict = {}
for character in characters:
  num_images = len(os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset/' 
                              + character))
  character_observations_dict[character] = num_images
  
listofTuples = sorted(character_observations_dict.items(), reverse=True,
                      key=lambda x: x[1])
 
# Iterate over the sorted sequence
for elem in listofTuples :
    print("{0:<26}{1:^2}{2:>13}".format(elem[0],"::", str(elem[1]) + " images."))

There are 42 characters.
homer_simpson             :: 2246 images.
ned_flanders              :: 1454 images.
moe_szyslak               :: 1452 images.
lisa_simpson              :: 1354 images.
bart_simpson              :: 1342 images.
marge_simpson             :: 1291 images.
krusty_the_clown          :: 1206 images.
principal_skinner         :: 1194 images.
charles_montgomery_burns  :: 1193 images.
milhouse_van_houten       :: 1079 images.
chief_wiggum              ::  986 images.
abraham_grampa_simpson    ::  913 images.
sideshow_bob              ::  877 images.
apu_nahasapeemapetilon    ::  623 images.
kent_brockman             ::  498 images.
comic_book_guy            ::  469 images.
edna_krabappel            ::  457 images.
nelson_muntz              ::  358 images.
lenny_leonard             ::  310 images.
mayor_quimby              ::  246 images.
waylon_smithers           ::  181 images.
maggie_simpson            ::  128 images.
groundskeeper_willie      ::  121 images.
barney_gu

## Searching for more data
As we can see in the above, some characters like Homer, Ned, Moe etc. have more than 1000 images whereas Lionel Hutz and Troy McClure have less then 10.

Ideally we want to balance the number of images per class (character) somewhat and so we will first get rid of a few characters which have a low number of images. Then we will use the `google_images_download` package to search for more images for some characters.

In [76]:
character_observations_sub_dict = {}

for character, num_images in character_observations_dict.items():
  if num_images >= 500:
    character_observations_sub_dict[character] = num_images
  else:
    shutil.rmtree('/content/simpsons/simpsons_dataset/simpsons_dataset/' + character)

characters = os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset')
print('There are now ' + str(len(characters)) + ' characters.')
    
listofTuples = sorted(character_observations_sub_dict.items(), reverse=True,
                      key=lambda x: x[1])
 
# Iterate over the sorted sequence
for elem in listofTuples :
    print("{0:<26}{1:^2}{2:>13}".format(elem[0],"::", str(elem[1]) + " images."))


There are now 14 characters.
homer_simpson             :: 2246 images.
ned_flanders              :: 1454 images.
moe_szyslak               :: 1452 images.
lisa_simpson              :: 1354 images.
bart_simpson              :: 1342 images.
marge_simpson             :: 1291 images.
krusty_the_clown          :: 1206 images.
principal_skinner         :: 1194 images.
charles_montgomery_burns  :: 1193 images.
milhouse_van_houten       :: 1079 images.
chief_wiggum              ::  986 images.
abraham_grampa_simpson    ::  913 images.
sideshow_bob              ::  877 images.
apu_nahasapeemapetilon    ::  623 images.


In [65]:
pip install google_images_download

Finding the mean of the number of images we have for each character, not including Homer.

In [81]:
i = 0
n = 0
for character, num_images in character_observations_sub_dict.items():
  if character != 'homer_simpson':
    i += num_images
    n += 1

mean = i/n
print('Mean =', mean)

Mean = 1151.076923076923


Downloading an extra 100 images (current limit of `google_images_download` without extra configuration) for all characters that have less then the mean number of images found above.



In [82]:
from google_images_download import google_images_download   #importing the library

response = google_images_download.googleimagesdownload()   #class instantiation

for character, num_images in character_observations_sub_dict.items():
  if num_images < mean:
    character_split = character.replace("_"," ")

    arguments_content = {"keywords":"simpsons " + character_split,
                         "limit": 100,
                         "output_directory":"/content/simpsons/simpsons_dataset/simpsons_dataset/",
                         "image_directory":character,
                         "format":"jpg"}   #creating dict of arguments
    paths = response.download(arguments_content)   #passing the arguments to the function


Item no.: 1 --> Item name = simpsons chief wiggum
Evaluating...
Starting Download...
Wrong image format returned. Skipping...
Completed Image ====> 1.hqdefault.jpg
Completed Image ====> 2.220px-Chief_Wiggum_Springfield_U_S_A.JPG
Completed Image ====> 3.81tYIp9qkGL._SY606_.jpg
Completed Image ====> 4.41igKzQTiDL.jpg
Completed Image ====> 5.BXNlzdFIIAANXII.jpg
Completed Image ====> 6.the-simpsons-thumb.jpg
Completed Image ====> 7.Chief%2BWiggum.jpg
Completed Image ====> 8.hqdefault.jpg
Completed Image ====> 9.Simpsons_06_21_P1_640x360_312568387694.jpg
Completed Image ====> 10.204f0014d0627cc76b67ae51e03f4047.jpg
Completed Image ====> 11.Simpsons_07_02_P3_640x360_321202243949.jpg
Completed Image ====> 12.chief-clancy-wiggum-from-the-simpsons-edward-fielding.jpg
Completed Image ====> 13.maxresdefault.jpg
Completed Image ====> 14.410UZH-yk1L.jpg
Completed Image ====> 15.chief-clancy-wiggum-the-simpsons-movie-64.4.jpg
Completed Image ====> 16.chief-wiggum-the-simpsons.jpg
Completed Image ==

Now we wish to store the images in a directory structure such that they can be used with the `ImageDataGenerator flow_from_directory`.

The below function will help us transfer data from a directory and split it into two different directories (a training and validation set) randomly and at a ratio you specify.

In [0]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  files_list = os.listdir(SOURCE)
  files_list = random.sample(files_list, len(files_list))
  
  for file in files_list:
    file_path = SOURCE + file
    if os.path.getsize(file_path) <= 0:
      files_list.remove(file)
      print(file + " is zero length, so ignoring.") 

  n = int(SPLIT_SIZE * len(files_list))
  
  for file in files_list[0:n]:
    shutil.copyfile(SOURCE + file, TRAINING + file)
      
  for file in files_list[n:]:
    shutil.copyfile(SOURCE + file, TESTING + file)
 

In [0]:
# Creating the directory structure for ImageDataGenerator
base_dir = '/content/simpsons_generator'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(train_dir)
os.mkdir(validation_dir)

characters = os.listdir('/content/simpsons/simpsons_dataset/simpsons_dataset')
split_size = .9

for character in characters:
  char_train_dir = os.path.join(train_dir, character)
  char_vald_dir = os.path.join(validation_dir, character)
  os.mkdir(char_train_dir)
  os.mkdir(char_vald_dir)
  
  source_dir = '/content/simpsons/simpsons_dataset/simpsons_dataset/' + character + '/'
  
  split_data(source_dir, char_train_dir + '/', char_vald_dir + '/', split_size)

We also wish to split the contents of the test data in /content/simpsons/kaggle_simpson_testset/kaggle_simpson_testset into a "testing" folder in our new directory.

**START HERE**
- Need to check if image is in `character_observations_sub_dict` and if it is then transfer it to the correct folder.

In [113]:
images_list = os.listdir('/content/simpsons/kaggle_simpson_testset/kaggle_simpson_testset/')
print(len(images_list))
for image in images_list:
  if image.find('abraham_grampa_simpson') != -1:
    print(image)

990
abraham_grampa_simpson_12.jpg
abraham_grampa_simpson_13.jpg
abraham_grampa_simpson_36.jpg
abraham_grampa_simpson_31.jpg
abraham_grampa_simpson_10.jpg
abraham_grampa_simpson_38.jpg
abraham_grampa_simpson_27.jpg
abraham_grampa_simpson_9.jpg
abraham_grampa_simpson_46.jpg
abraham_grampa_simpson_34.jpg
abraham_grampa_simpson_8.jpg
abraham_grampa_simpson_3.jpg
abraham_grampa_simpson_49.jpg
abraham_grampa_simpson_19.jpg
abraham_grampa_simpson_42.jpg
abraham_grampa_simpson_39.jpg
abraham_grampa_simpson_16.jpg
abraham_grampa_simpson_32.jpg
abraham_grampa_simpson_7.jpg
abraham_grampa_simpson_28.jpg
abraham_grampa_simpson_1.jpg
abraham_grampa_simpson_43.jpg
abraham_grampa_simpson_18.jpg
abraham_grampa_simpson_47.jpg
abraham_grampa_simpson_26.jpg
abraham_grampa_simpson_29.jpg
abraham_grampa_simpson_44.jpg
abraham_grampa_simpson_2.jpg
abraham_grampa_simpson_48.jpg
abraham_grampa_simpson_21.jpg
abraham_grampa_simpson_25.jpg
abraham_grampa_simpson_33.jpg
abraham_grampa_simpson_0.jpg
abraham_gramp

# Saving to Google Drive

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
zipname = 'simpsons_generator_directory'

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

zipfolder(zipname, '/content/simpsons/simpsons_dataset/simpsons_dataset/fat_tony')

# 2. Create & upload a file text file.
file1 = drive.CreateFile()
file1.SetContentFile(zipname+".zip")
file1.Upload()

* Split data into training and testing folders
* Define a model
* Use something like 

`TRAINING_DIR = "/tmp/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))`


In [0]:
"""
Use this code to delete a directory is required
"""
import shutil

shutil.rmtree('/content/simpsons/training')